In [1]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [108]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/숨고/임지원님(날씨 예측)')

weather = pd.read_csv('해운대 날씨.csv')
customer = pd.read_csv('해운대 입장객수.csv')

In [109]:
weather.head(5)

,날짜,강수_관측값,기온,습도,체감온도,평균풍속,평균기압,평균수온,평균최대파고,평균파주기
0,2012-05-01,10.5,20.4,62.1,21.5,6.5,1009.0,15.2,1.2,4.8
1,2012-05-02,1.0,19.6,70.5,21.4,7.8,1004.0,15.5,2.1,5.7
2,2012-05-03,0.1,20.7,77.5,22.9,4.1,1004.0,16.4,1.9,6.6
3,2012-05-04,0.0,19.9,70.7,21.7,4.1,1006.5,16.4,1.6,7.5
4,2012-05-05,0.0,24.8,47.2,24.7,6.1,1004.9,15.8,1.9,7.8


In [110]:
customer.head(5)

,방문일,방문객수(명)
0,2018-06-01,"28,423"
1,2018-06-02,"29,540"
2,2018-06-03,"28,603"
3,2018-06-04,"27,336"
4,2018-06-05,"37,173"


In [112]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers import Bidirectional
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import time
import inspect

# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)

customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수(명)'] = customer['방문객수(명)'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)

In [113]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1618 entries, 0 to 1617
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1618 non-null   datetime64[ns]
 1   강수_관측값  1618 non-null   float64       
 2   기온      1618 non-null   float64       
 3   습도      1618 non-null   float64       
 4   체감온도    1618 non-null   float64       
 5   평균풍속    1567 non-null   float64       
 6   평균기압    1557 non-null   float64       
 7   평균수온    1511 non-null   float64       
 8   평균최대파고  1552 non-null   float64       
 9   평균파주기   1551 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 126.5 KB


In [114]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   방문일      332 non-null    datetime64[ns]
 1   방문객수(명)  332 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.3 KB


In [115]:
# merge data : 방문객수 예측을 위한 데이터
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기", "방문객수(명)"]]

# weather data : 날씨 예측을 위한 데이터(na는 제거하는 방향으로 일단 진행)
weather = weather.dropna()

## 방문객 수 예측

In [116]:
# 방문객 수 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(total_data.iloc[:, :-1], total_data['방문객수(명)'], test_size=0.2)

In [117]:
# 방문객 수 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [ ]:
# 방문객 수 예측을 위한  model training
model=Sequential()
model.add(Dense(256,input_dim=9, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 256)               2560      
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 1)               

In [ ]:
# 학습 진행
earlyStopping=EarlyStopping(monitor='val_loss',patience=10,mode='min',verbose=1,restore_best_weights=True) 
model.compile(loss='mse',optimizer='adam')
hist=model.fit(train_x_scaled,train_y,epochs=500,batch_size=16,validation_split=0.1, callbacks=[earlyStopping])

In [ ]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 입장객 수: ', pred_y[-1:])

2/2 [==============================] - 0s 11ms/step - loss: 3316367872.0000
loss:  3316367872.0
예상 입장객 수:  [[61907.344]]


## 강수_관측값 예측

In [118]:
# 사용 데이터
weather.info()

# input data
weather_input = weather[["기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기"]]
weather_output = weather[['강수_관측값']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1486 entries, 0 to 1617
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1486 non-null   datetime64[ns]
 1   강수_관측값  1486 non-null   float64       
 2   기온      1486 non-null   float64       
 3   습도      1486 non-null   float64       
 4   체감온도    1486 non-null   float64       
 5   평균풍속    1486 non-null   float64       
 6   평균기압    1486 non-null   float64       
 7   평균수온    1486 non-null   float64       
 8   평균최대파고  1486 non-null   float64       
 9   평균파주기   1486 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 127.7 KB


In [119]:
# 강수_관측값 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input.values, weather_output.values, test_size=0.2)

In [120]:
# 강수_관측값 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [121]:
# 강수_관측값 예측을 위한 data reshape
train_x_scaled = train_x_scaled.reshape(-1, 1, 8)
train_x_scaled = train_x_scaled.astype(float)
test_x_scaled = test_x_scaled.reshape(-1, 1, 8)
test_x_scaled = test_x_scaled.astype(float)

In [122]:
train_y = train_y.astype(float)
test_y = test_y.astype(float)

In [137]:
# 강수_관측값 예측을 위한 lstm 학습
model=Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(1,8)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 1, 128)            70144     
                                                                 
 dense_13 (Dense)            (None, 1, 32)             4128      
                                                                 
 dense_14 (Dense)            (None, 1, 1)              33        
                                                                 
Total params: 74,305
Trainable params: 74,305
Non-trainable params: 0
_________________________________________________________________


In [139]:
# 학습 진행
earlyStopping = EarlyStopping(monitor='val_loss', patience=30, mode='min', verbose=1,restore_best_weights=True)
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
hist = model.fit(train_x_scaled, train_y, epochs=500, batch_size=32, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

Epoch 1/500
30/30 - 5s - loss: 412.6611 - val_loss: 407.9982 - 5s/epoch - 153ms/step
Epoch 2/500
30/30 - 0s - loss: 406.0373 - val_loss: 400.8677 - 117ms/epoch - 4ms/step
Epoch 3/500
30/30 - 0s - loss: 399.2627 - val_loss: 393.6810 - 128ms/epoch - 4ms/step
Epoch 4/500
30/30 - 0s - loss: 392.1417 - val_loss: 385.1298 - 120ms/epoch - 4ms/step
Epoch 5/500
30/30 - 0s - loss: 384.9869 - val_loss: 378.1822 - 122ms/epoch - 4ms/step
Epoch 6/500
30/30 - 0s - loss: 379.7222 - val_loss: 369.6219 - 131ms/epoch - 4ms/step
Epoch 7/500
30/30 - 0s - loss: 371.8124 - val_loss: 363.2657 - 127ms/epoch - 4ms/step
Epoch 8/500
30/30 - 0s - loss: 366.9465 - val_loss: 355.6311 - 129ms/epoch - 4ms/step
Epoch 9/500
30/30 - 0s - loss: 361.2654 - val_loss: 348.8982 - 138ms/epoch - 5ms/step
Epoch 10/500
30/30 - 0s - loss: 358.5368 - val_loss: 347.4610 - 176ms/epoch - 6ms/step
Epoch 11/500
30/30 - 0s - loss: 352.4493 - val_loss: 343.2360 - 202ms/epoch - 7ms/step
Epoch 12/500
30/30 - 0s - loss: 347.5667 - val_loss: 

In [136]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 강수_관측값 : ', pred_y[-1:])

10/10 [==============================] - 0s 5ms/step - loss: 444.6830
loss:  444.6829833984375
예상 강수_관측값 :  [[[0.84299386]]]


## 기온 예측

In [140]:
# 사용 데이터
weather.info()

# input data
weather_input = weather[["강수_관측값", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기"]]
weather_output = weather[['기온']]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1486 entries, 0 to 1617
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      1486 non-null   datetime64[ns]
 1   강수_관측값  1486 non-null   float64       
 2   기온      1486 non-null   float64       
 3   습도      1486 non-null   float64       
 4   체감온도    1486 non-null   float64       
 5   평균풍속    1486 non-null   float64       
 6   평균기압    1486 non-null   float64       
 7   평균수온    1486 non-null   float64       
 8   평균최대파고  1486 non-null   float64       
 9   평균파주기   1486 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 127.7 KB


In [141]:
# 기온 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input.values, weather_output.values, test_size=0.2)

In [142]:
# 기온 예측을 위한  minmax scaler
x_mm_scaler = MinMaxScaler()
train_x_scaled = x_mm_scaler.fit_transform(train_x)
test_x_scaled = x_mm_scaler.transform(test_x)

In [145]:
# 기온 예측을 위한 data reshape
train_x_scaled = train_x_scaled.reshape(-1, 1, 8)
test_x_scaled = test_x_scaled.reshape(-1, 1, 8)

In [146]:
# 기온 예측을 위한 lstm 학습
model=Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(1,8)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 1, 128)            70144     
                                                                 
 dense_17 (Dense)            (None, 1, 32)             4128      
                                                                 
 dense_18 (Dense)            (None, 1, 1)              33        
                                                                 
Total params: 74,305
Trainable params: 74,305
Non-trainable params: 0
_________________________________________________________________


In [150]:
# 학습 진행
earlyStopping = EarlyStopping(monitor='val_loss', patience=30, mode='min', verbose=1,restore_best_weights=True)
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
hist = model.fit(train_x_scaled, train_y, epochs=500, batch_size=64, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

Epoch 1/500
15/15 - 3s - loss: 0.0456 - val_loss: 0.0318 - 3s/epoch - 209ms/step
Epoch 2/500
15/15 - 0s - loss: 0.0160 - val_loss: 0.0188 - 127ms/epoch - 8ms/step
Epoch 3/500
15/15 - 0s - loss: 0.0095 - val_loss: 0.0144 - 137ms/epoch - 9ms/step
Epoch 4/500
15/15 - 0s - loss: 0.0093 - val_loss: 0.0151 - 128ms/epoch - 9ms/step
Epoch 5/500
15/15 - 0s - loss: 0.0090 - val_loss: 0.0156 - 147ms/epoch - 10ms/step
Epoch 6/500
15/15 - 0s - loss: 0.0094 - val_loss: 0.0161 - 123ms/epoch - 8ms/step
Epoch 7/500
15/15 - 0s - loss: 0.0098 - val_loss: 0.0150 - 120ms/epoch - 8ms/step
Epoch 8/500
15/15 - 0s - loss: 0.0094 - val_loss: 0.0142 - 121ms/epoch - 8ms/step
Epoch 9/500
15/15 - 0s - loss: 0.0094 - val_loss: 0.0144 - 131ms/epoch - 9ms/step
Epoch 10/500
15/15 - 0s - loss: 0.0093 - val_loss: 0.0147 - 128ms/epoch - 9ms/step
Epoch 11/500
15/15 - 0s - loss: 0.0094 - val_loss: 0.0145 - 122ms/epoch - 8ms/step
Epoch 12/500
15/15 - 0s - loss: 0.0094 - val_loss: 0.0142 - 138ms/epoch - 9ms/step
Epoch 13/500


In [151]:
loss=model.evaluate(test_x_scaled,test_y)
pred_y=model.predict(test_x_scaled)

print('loss: ',loss)
print('예상 기온 예측값 : ', pred_y[-1:])

10/10 [==============================] - 0s 3ms/step - loss: 0.0106
loss:  0.010564546100795269
예상 기온 예측값 :  [[[25.737276]]]
